In [1]:
import sklearn
#from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier #assuming yas wants
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score

import os
import pandas as pd
import numpy as np

from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

import pickle #to save/load trained classifiers

In [2]:
#Where are the files
file_data = '..' + os.sep + 'data' + os.sep + 'okapi_data_metadata.csv'
df = pd.read_csv(file_data)
label = np.array(df['diagnosis'])

# Creating a binary array corresponding to cancer/non-cancer (1 = cancer)
cancer = np.copy(label)
cancer[cancer == "BCC"] = 1
cancer[cancer == "MEL"] = 1
cancer[cancer == "SCC"] = 1
cancer[cancer == "ACK"] = 0
cancer[cancer == "NEV"] = 0
cancer[cancer == "SEK"] = 0

In [3]:
#create a np array from the generated features scores
annotation_data = '..' + os.sep + 'features' + os.sep + 'features_okapi_images.csv'
our_annotations = pd.read_csv(annotation_data)
scores_per_image = []

for i in range(len(our_annotations)):
    scores_per_image.append((our_annotations["asymmetry"][i], our_annotations["colour"][i], our_annotations["dots"][i]))

correct_scores = np.array(scores_per_image)

In [4]:
#load our data set as a dictionary, the feature scores need to go in the "data" key
#EQUIVALENT TO A FOLDER SYSTEM,
#WHERE the key named "DATA" IS WHERE THE FEATURE ANNOTATIONS ARE KEPT
#"FEATURE_NAMES" ARE WHERE THE COLUMN NAMES ARE KEPT FOR REFERENCE
#
features_csv = {"data": None, "class": None, "class_names": None, "feature_names": None}
print(features_csv)

features_csv["data"] = correct_scores

our_feature_names = ["Asymmetry", "Colour_count", "Dots_and_Globules"]
features_csv["feature_names"] = our_feature_names

our_classes = ["Not_Cancer", "Cancer"]

features_csv["class_names"] = our_classes
features_csv["class"] = cancer


{'data': None, 'class': None, 'class_names': None, 'feature_names': None}


### ONCE DATA IS READY IN A TABULAR FORMAT, WE NEED TO SPLIT IT, EITHER:
 - TRAIN, TEST SPLIT
 - TRAIN, VALIDATION, TEST SPLIT'
 - CROSS-VALIDATION
 - ETC

In [5]:
#KFolding prep
num_folds = 5
kfold = KFold(n_splits =  num_folds)

In [6]:
#initialises our decision tree using the imported data structure
our_dec_tree = DecisionTreeClassifier()

#intialises our KNN classifier
our_k_NN = DecisionTreeClassifier() #This is to be changed YASMINE

#Classifier list to be iterated through later
list_of_classifiers = [our_dec_tree, our_k_NN]

In [10]:
#the initial setting aside of test data for the final selected model after validation selects a winner
x_entire_train, x_test, y_entire_train, y_test = train_test_split(features_csv["data"], features_csv["class"])
print(x_entire_train.shape)
print(x_test.shape)
print(y_entire_train.shape)
print(y_test.shape)

(79, 3)
(27, 3)
(79,)
(27,)


In [11]:
#Initialise a np array for the classifier performance scores
acc_val = np.empty([num_folds,len(list_of_classifiers)])


#for loops runs the same num_folds we have chosen
for i_data, (train_index, val_index) in enumerate(kfold.split(x_entire_train,y_entire_train)):
    #the number of training data sets created will therefore also be according to the num_folds
    x_train = x_entire_train[train_index,:]
    y_train = y_entire_train[train_index]
    x_val = x_entire_train[val_index,:]
    y_val = y_entire_train[val_index]

    #within each for loop (for each set of training/validation data)
    #we'll run the data on each of our classifiers

    for j, clf in enumerate(list_of_classifiers):

        #Train the classifier
        clf.fit(x_train,y_train)
    
        #Trying out balanced_accuracy_score here as it will balance based upon the proportion of cancer/not cancer in truth
        acc_val[i,j] = balanced_accuracy_score(y_val, clf.predict(x_val))

ValueError: Unknown label type: unknown. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [ ]:
#This sets aside some data, designated the test data (for our final test once we've picked a model to use)
X_train, X_test, y_train, y_test = train_test_split(features_csv["data"], features_csv["class"])

#This is to illustrate the split amounts (the first is a sum of the others)
print(features_csv["data"].shape)
print(X_train.shape)
print(X_test.shape)
print(y_train)
print(y_test)

In [ ]:
#make predictions of the class of the test data
y1_pred = our_dec_tree.predict(X1_test)

In [ ]:
#returns the fraction of correctly classified samples (0-1) scale
accuracy_score(y1_test, y1_pred)
#where y1_test is the groundtruth, and y1_pred is the model prediction
#can give count of correct if normalize = False

In [ ]:
#seems straightforward
plot_tree(our_dec_tree)